In [1]:
import pandas as pd
import re
from bs4 import BeautifulSoup
from langdetect import detect as detect_language

In [2]:
p = 'combined_agency_data_June_27.csv'

In [3]:
df = pd.read_csv(p)

# Remove empty textstring cells

In [4]:
cleaned_df = df[df.Textstring.notnull()]

# Removing html tags

In [5]:
def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext

In [6]:
cleaned_df.Textstring = cleaned_df.Textstring.apply(lambda x : cleanhtml(x))

/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [7]:
untagged_df = cleaned_df

# Detect language of the text

In [8]:
def detect_language_(s):
    try:
        language = detect_language(s)
        return language
    except:
        return ''

In [9]:
untagged_df['language'] = untagged_df.Textstring.apply(lambda x : detect_language_(x))

/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [10]:
augmented_df = untagged_df

# Remove unlabelled data points

In [11]:
labelled_df = augmented_df[augmented_df['Comment Ala'].notnull()]

# Find forbidden data points

In [12]:
for i in range(len(labelled_df)):
    content = labelled_df.Textstring.iloc[i]
    if 'forbidden' in content:
        labelled_df.loc[i, 'drop'] = True
    else:
        labelled_df.loc[i, 'drop'] = False

/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/home/nick/.pyenv/versions/agency/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame


In [13]:
valid_points_df = labelled_df[labelled_df['drop']== False]

In [14]:
del valid_points_df['drop']

# Rephrase label

In [15]:
def rename_label(label):
    if isinstance(label, float) or label.lower() == 'no agency':
        return False
    else:
        return True

In [16]:
valid_points_df = valid_points_df.rename(columns={'Comment Ala': 'label'})

In [17]:
valid_points_df.label = valid_points_df.label.apply(lambda x : rename_label(x))

# Restrict to english language

In [18]:
df_en = valid_points_df[valid_points_df.language=='en']

In [19]:
df_en.to_csv('agency_data_en.csv')

In [20]:
df_en.label.value_counts()

False    898
True     715
Name: label, dtype: int64

In [21]:
df_en.head()

,Unnamed: 0,Textstring,label,Domain,language
0,0.0,skip to content toggle navigation ...,True,http://cleverreach.com,en
1,1.0,safedome home products popular products rechar...,True,http://maxwellforest.com,en
2,2.0,skip to main navigation logo ...,True,http://spark.net,en
3,3.0,top sign in creat...,True,http://ab.com,en
4,4.0,skip main navigation menu marri...,True,http://marriot.com,en
